## Lab 10 - Clasificación de Perros y Gatos

Stefano Aragoni, Carol Arévalo, Luis Diego Santos

### Importar Librerías


In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
import cv2
import random
import tensorflow as tf
from PIL import Image
from skimage import color, filters, measure

2023-05-03 16:14:08.150196: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


## 1.1 Lectura del Dataset

### Lectura y Procesamiento de Imagenes

In [2]:
def is_image_corrupted(file_path):
    try:
        with Image.open(file_path) as img:
            img.verify()
        return False
    except:
        return True

In [13]:
def load_images(path, size=(150, 150)):
    images = []
    labels = []

    dirs = ["Cat", "Dog"]

    for dir in dirs:
        elements = os.listdir(path+'/'+dir)
        random.shuffle(elements)

        contador = 0

        for file in elements:
            if file.endswith('.jpg'):
                img_path = os.path.join(path+'/'+dir, file)
                label = 1 if 'Dog' == dir else 0

                corrupta = is_image_corrupted(img_path)

                if corrupta:
                    continue

                # Leer imagen a color
                img = cv2.imread(img_path)

                try:
                    # Redimensionar imagen
                    img = cv2.resize(img, size)

                    # Convertir imagen a escala de grises
                    gray_img = color.rgb2gray(img)

                    # Aplicar umbral adaptativo para binarizar imagen
                    binary_img = gray_img > filters.threshold_local(gray_img, block_size=35)

                    # Label the objects in the binary image
                    labeled_img = measure.label(binary_img)

                    # Find the properties of the labeled objects
                    properties = measure.regionprops(labeled_img)

                    # Find the object with the largest area
                    max_area = 0
                    max_area_index = 0
                    for i, prop in enumerate(properties):
                        if prop.area > max_area:
                            max_area = prop.area
                            max_area_index = i

                    # Extract the coordinates of the bounding box of the object
                    min_row, min_col, max_row, max_col = properties[max_area_index].bbox

                    # Crop the image to the bounding box of the object
                    img = img[min_row:max_row, min_col:max_col]

                    # expand the image to 150x150
                    img = cv2.resize(img, size)

                    # Girar imagen aleatoriamente
                    if random.random() < 0.5:
                        opt = random.randint(0, 3)

                        if opt == 0:
                            img = cv2.flip(img, 1)

                        elif opt == 1:
                            img = cv2.rotate(img, cv2.ROTATE_180)

                        elif opt == 2:
                            img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)

                        elif opt == 3:
                            img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)


                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    
                    # Normalizar valores de píxeles
                    img = img / 255.0

                    # # show image on popup
                    # cv2.imshow('image', img)
                    # cv2.waitKey(0)
                    # cv2.destroyAllWindows()

                    images.append(img)
                    labels.append(label)
                            
                    if contador == 1300:
                        break

                    if contador % 100 == 0:
                        print("Imagenes procesadas: ", contador)  

                    contador += 1

                except Exception as e:
                    pass

    return np.array(images), np.array(labels)

### Importar Imagenes y Split de Dataset

In [14]:
# Definir ruta del conjunto de datos
data_path = './PetImages'

# Cargar imágenes y etiquetas
images, labels = load_images(data_path)

# Comprobar tamaños de los conjuntos
print('\nNúmero de imágenes por clase:', np.bincount(labels))

Imagenes procesadas:  0
Imagenes procesadas:  100
Imagenes procesadas:  200
Imagenes procesadas:  300
Imagenes procesadas:  400
Imagenes procesadas:  500
Imagenes procesadas:  600
Imagenes procesadas:  700
Imagenes procesadas:  800
Imagenes procesadas:  900
Imagenes procesadas:  1000
Imagenes procesadas:  1100
Imagenes procesadas:  1200
Imagenes procesadas:  0
Imagenes procesadas:  100
Imagenes procesadas:  200
Imagenes procesadas:  300
Imagenes procesadas:  400
Imagenes procesadas:  500
Imagenes procesadas:  600
Imagenes procesadas:  700
Imagenes procesadas:  800
Imagenes procesadas:  900
Imagenes procesadas:  1000
Imagenes procesadas:  1100
Imagenes procesadas:  1200

Número de imágenes por clase: [1301 1301]


In [15]:
# Dividir conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Dividir 20 de prueba en 10 de validación y 10 de prueba
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# Verificar tamaños de los conjuntos
print('\nTamaño del conjunto de entrenamiento:', X_train.shape)
print('Tamaño del conjunto de validación:', X_val.shape)
print('Tamaño del conjunto de prueba:', X_test.shape)


Tamaño del conjunto de entrenamiento: (2081, 150, 150)
Tamaño del conjunto de validación: (261, 150, 150)
Tamaño del conjunto de prueba: (260, 150, 150)


In [16]:
from keras.utils import to_categorical

y_train = to_categorical(y_train, 2)
y_val = to_categorical(y_val, 2)

## 1.2 Construccion del Modelo

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
          epochs=20,
          verbose=1,
          validation_data=(X_val, y_val))

Epoch 1/20
66/66 [==============================] - 59s 847ms/step - loss: 1.2153 - accuracy: 0.5435 - val_loss: 1.7346 - val_accuracy: 0.5172
Epoch 2/20
66/66 [==============================] - 55s 833ms/step - loss: 0.8630 - accuracy: 0.5805 - val_loss: 0.7236 - val_accuracy: 0.4943
Epoch 3/20
66/66 [==============================] - 51s 764ms/step - loss: 0.7625 - accuracy: 0.6358 - val_loss: 0.7903 - val_accuracy: 0.5019
Epoch 4/20
66/66 [==============================] - 50s 752ms/step - loss: 0.6828 - accuracy: 0.6564 - val_loss: 0.6948 - val_accuracy: 0.5134
Epoch 5/20
66/66 [==============================] - 51s 765ms/step - loss: 0.6479 - accuracy: 0.6756 - val_loss: 0.8405 - val_accuracy: 0.5211
Epoch 6/20
66/66 [==============================] - 53s 808ms/step - loss: 0.5974 - accuracy: 0.7136 - val_loss: 0.7336 - val_accuracy: 0.5632
Epoch 7/20
66/66 [==============================] - 55s 837ms/step - loss: 0.5630 - accuracy: 0.7213 - val_loss: 0.8610 - val_accuracy: 0.5517